In [1]:
import datetime
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import *

In [2]:
def load_data(type_text):
    f_data = open('../emnist/emnist-byclass-'+ type_text + '-images-idx3-ubyte')
    f_data = np.fromfile(file=f_data, dtype=np.uint8)
    f_data = f_data[16:].reshape(-1, 28, 28).astype(np.uint8)
    f_label = open('../emnist/emnist-byclass-'+ type_text + '-labels-idx1-ubyte')
    f_label = np.fromfile(file=f_label, dtype=np.uint8)
    f_label = f_label[8:].reshape(-1).astype(np.uint8)
    return f_data, f_label

In [3]:
train_data, train_label = load_data('train')
test_data, test_label = load_data('test')

In [4]:
# Validation data size: Train=697,932, Test=116,323
print("Train data size: {train}, Test data size: {test}".format(train=len(train_data), test=len(test_data)))

Train data size: 697932, Test data size: 116323


In [5]:
train_label = to_categorical(train_label, num_classes=62, dtype='uint8')
test_label = to_categorical(test_label, num_classes=62, dtype='uint8')

In [6]:
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
# Reshape Training Data
train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))

# Reshape Testing Data
test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

In [8]:
train_data.shape

(697932, 28, 28, 1)

In [9]:
train_label.shape

(697932, 62)

In [ ]:
# create model
model = Sequential()

# =====add layer=====
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(62, activation = "softmax"))
# ===================

# # Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
plot_model(model_3, to_file="image/baseline_model.png", show_shapes=True)

In [ ]:
# Baseline: Test Data Accuracy = 0.87
model.fit(train_data, train_label, epochs=20, batch_size=512, verbose=2, shuffle=True, validation_split=0.1, callbacks=[tensorboard_callback])

In [ ]:
results = model.evaluate(test_data, test_label)

In [ ]:
model.save('saved_model/baseline_model')

In [ ]:
# create model
model_2 = Sequential()

# =====add layer=====
model_2.add(Conv2D(6, kernel_size=5, strides=1, padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model_2.add(BatchNormalization())

model_2.add(Conv2D(16, kernel_size=5, strides=1, padding = 'Same', activation ='relu'))
model_2.add(BatchNormalization())

model_2.add(AveragePooling2D(pool_size=2, strides=2, padding='valid'))
model_2.add(Dropout(0.3))

model_2.add(Conv2D(32, kernel_size=5, strides=1, padding = 'Same', activation ='relu'))
model_2.add(BatchNormalization())

model_2.add(Conv2D(64, kernel_size=5, strides=1, padding = 'Same', activation ='relu'))
model_2.add(BatchNormalization())
model_2.add(AveragePooling2D(pool_size=2, strides=2, padding='valid'))
model_2.add(Dropout(0.3))

model_2.add(Conv2D(128, kernel_size=5, strides=1, padding = 'Same',  activation ='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.3))

model_2.add(Flatten())

model_2.add(Dense(256, activation = "relu"))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.25))

model_2.add(Dense(62, activation = "softmax"))
# ===================

# # Compile model
opt = Adam(learning_rate=0.005)
model_2.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
model_2.summary()
plot_model(model_3, to_file="image/improvement_1_model.png", show_shapes=True)

In [ ]:
model_2.fit(train_data, train_label, epochs=30, verbose=2, batch_size=512, shuffle=True, validation_split=0.1, callbacks=[tensorboard_callback])

In [ ]:
model_2.evaluate(test_data, test_label)

In [ ]:
model_2.save('saved_model/improvement_model_1')

In [ ]:
# load model
model_2 = tf.keras.models.load_model('saved_model/improvement_1_model')

In [ ]:
results = model_2.evaluate(test_data, test_label)

In [ ]:
pred_Y = model_2.predict(test_data)

In [ ]:
# DRAW confusion-matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pred_Y = np.argmax(pred_Y, axis=1)

In [ ]:
true_Y = np.argmax(test_label, axis=1)

In [ ]:
conf_matrix = confusion_matrix(true_Y, pred_Y)

In [ ]:
import string

axis_labels = [] # labels for axis

# push 0~9
for d in string.digits:
    axis_labels.append(d)

# push A~Z
for uc in string.ascii_uppercase:
    axis_labels.append(uc)

# push a~z
for ul in string.ascii_lowercase:
    axis_labels.append(ul)

In [ ]:
fig, ax = plt.subplots(figsize=(80,60))         # Sample figsize in inches
fig = sns.heatmap(conf_matrix, annot=True, fmt='d', linewidths=.5, ax=ax, xticklabels=axis_labels, yticklabels=axis_labels)

In [ ]:
fig = fig.get_figure()

In [ ]:
fig.savefig("image/confusion_matrix.png")

In [ ]:
# start from 0 to 61
# 0,1,2,3,4,5,6,7,8,9,A...Z,a...Z